## PROMPT
Generate a pipeline for Databricks
* Get Data information from two web addresses:
   * Space launches: https://api.spacexdata.com/v3/launches
   * All rockets: https://api.spacexdata.com/v3/rockets
* Do a join operation between the launches and the rockets
* Send the chosen information to the web address: https://httpbin.org/post
* The script must provide status updates on its progress, report any errors encountered, confirm the outcome of the final data sending step, and measure/report execution times.

In [0]:
import requests
import json
import time

# Helper function to log messages with timestamps
def log(message):
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {message}")

# Function to fetch data from a given URL
def fetch_data(url):
    try:
        log(f"Fetching data from: {url}")
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        log("Data fetched successfully.")
        return response.json()
    except requests.exceptions.RequestException as e:
        log(f"Error fetching data from {url}: {e}")
        raise

# Function to send data to the target URL
def send_data(url, data):
    try:
        log(f"Sending data to: {url}")
        response = requests.post(url, json=data)
        response.raise_for_status()  # Raise an exception for HTTP errors
        log("Data sent successfully.")
        return response.json()
    except requests.exceptions.RequestException as e:
        log(f"Error sending data to {url}: {e}")
        raise

# Main pipeline function
def main():
    start_time = time.time()

    # Step 1: Fetch data from the two web addresses
    try:
        launches_url = "https://api.spacexdata.com/v3/launches"
        rockets_url = "https://api.spacexdata.com/v3/rockets"

        launches_data = fetch_data(launches_url)
        rockets_data = fetch_data(rockets_url)

        log(f"Number of launches fetched: {len(launches_data)}")
        log(f"Number of rockets fetched: {len(rockets_data)}")

    except Exception as e:
        log(f"Pipeline failed during data fetching: {e}")
        return

    # Step 2: Perform a join operation between launches and rockets
    try:
        log("Performing join operation between launches and rockets...")
        rockets_dict = {rocket['id']: rocket for rocket in rockets_data}

        joined_data = []
        for launch in launches_data:
            rocket_id = launch.get('rocket', {}).get('rocket_id')
            rocket_info = rockets_dict.get(rocket_id, {})
            joined_entry = {
                'launch_name': launch.get('mission_name'),
                'launch_date': launch.get('launch_date_utc'),
                'rocket_name': rocket_info.get('name'),
                'rocket_country': rocket_info.get('country'),
                'rocket_description': rocket_info.get('description')
            }
            joined_data.append(joined_entry)

        log(f"Join operation completed. Total entries: {len(joined_data)}")

    except Exception as e:
        log(f"Pipeline failed during join operation: {e}")
        return

    # Step 3: Send the joined data to the target URL
    try:
        target_url = "https://httpbin.org/post"
        response = send_data(target_url, joined_data)
        log(f"Response from target URL: {json.dumps(response, indent=2)}")

    except Exception as e:
        log(f"Pipeline failed during data sending: {e}")
        return

    # Final step: Report execution time
    end_time = time.time()
    execution_time = end_time - start_time
    log(f"Pipeline completed successfully. Total execution time: {execution_time:.2f} seconds.")

# Run the pipeline
if __name__ == "__main__":
    main()